In [1]:
!pip install selenium beautifulsoup4

In [1]:
!pip install openpyxl

In [ ]:
!pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install selenium-wire

   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 3.2/3.2 MB 29.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install blinker==1.5

  Attempting uninstall: blinker
    Found existing installation: blinker 1.9.0
    Uninstalling blinker-1.9.0:
      Successfully uninstalled blinker-1.9.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import time
import pandas as pd
import subprocess
from urllib.parse import quote
from seleniumwire import webdriver  # selenium-wire 사용
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# API Key
API_KEY = "API키"

# 엑셀 경로 및 저장 폴더
excel_path = r"C:/Users/Administrator/Downloads/OpenDataCCTV.xlsx"
save_dir = os.path.join(os.getcwd(), "cctv_videos")
os.makedirs(save_dir, exist_ok=True)

# 엑셀 확인
if not os.path.exists(excel_path):
    raise FileNotFoundError(f"엑셀 파일을 찾을 수 없습니다: {excel_path}")

df = pd.read_excel(excel_path)

# 크롬 옵션 설정
options = Options()
options.add_argument("--mute-audio")
options.add_argument("--disable-gpu")
# options.add_argument("--headless")  # 필요 시 사용

driver = webdriver.Chrome(service=Service(), options=options)
wait = WebDriverWait(driver, 10)

# FFmpeg 경로 확인
ffmpeg_path = "ffmpeg"
if not subprocess.run([ffmpeg_path, "-version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE).returncode == 0:
    raise FileNotFoundError("FFmpeg이 설치되지 않았거나 경로가 잘못되었습니다.")

# 반복 처리
for idx, row in df.iterrows():
    try:
        cctvid = row["CCTVID"]
        cctvname = row.get("CCTVNAME", "")
        encoded_name = quote(cctvname)

        url = f"https://www.utic.go.kr/jsp/map/openDataCctvStream.jsp?key={API_KEY}&cctvid={cctvid}&cctvName={encoded_name}&kind=KB"

        print(f"[INFO] 접속 중: {cctvid}")
        driver.get(url)
        time.sleep(5)  # 로딩 대기

        # <source src="...mp4"> 추출
        source_element = driver.find_element(By.TAG_NAME, "source")
        video_url = source_element.get_attribute("src")

        if not video_url:
            print(f"[WARN] mp4 링크 없음: {cctvid}")
            continue

        if video_url.startswith("//"):
            video_url = "https:" + video_url  # 프로토콜 보완

        output_path = os.path.join(save_dir, f"{cctvid}.mp4")
        print(f"[INFO] 다운로드 시작: {video_url}")

        # FFmpeg로 10초 저장 (원하면 -t 값 조절)
        ffmpeg_cmd = [
            ffmpeg_path, "-y",
            "-i", video_url,
            "-t", "10",
            "-c", "copy",
            output_path
        ]

        result = subprocess.run(ffmpeg_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"[DEBUG] FFmpeg 출력: {result.stdout.decode()}")
        print(f"[DEBUG] FFmpeg 에러: {result.stderr.decode()}")

        if os.path.exists(output_path):
            print(f"[SUCCESS] 저장 완료: {output_path}")
        else:
            print(f"[ERROR] 저장 실패: {output_path}")

    except Exception as e:
        print(f"[ERROR] {cctvid} 처리 실패: {e}")

driver.quit()

C:\Users\Administrator\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


[INFO] 접속 중: L933113
[INFO] 다운로드 시작: https://cctvsec.ktict.co.kr/9952/D61TTHsCphQPMeaRy3lH90zv6QVJyU0KwH5r0DPkSPBs2e3yapDEn7sZan6zkkvC
[DEBUG] FFmpeg 출력: 
[DEBUG] FFmpeg 에러: ffmpeg version 7.1.1-essentials_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-dxva2 --enable-d3d11va --enabl